In [343]:
import numpy as np
from random import randint
import math
import unittest
import operator
import numbers
import sys

In [534]:
class TicTacToe:

    # There are 5812 legal board states that can be reached before there is a winner
    # http://brianshourd.com/posts/2012-11-06-tilt-number-of-tic-tac-toe-boards.html
    
    __bad_move_game_is_over = -1
    __bad_move_action_already_played = -2
    __bad_move_no_consecutive_plays = -3
    __good_move = 0 # value of a free action space on board
    __play = 0.001 # reward for playing an action
    __draw = 0.002 # reward for playing to end but no one wins
    __win = 0.01 # reward for winning a game
    __no_player = -2 # id of a non existent player i.e. used to record id of player that has not played
    __win_mask = np.full((1, 3),3,np.int8)
    actions =  {1:(0,0), 2:(0,1), 3:(0,2), 4:(1,0), 5:(1,1), 6:(1,2), 7:(2,0), 8:(2,1), 9:(2,2)}
    __num_actions = 9
    player_X = 1
    player_O = -1

    #
    # Return game to intial state, where no one has played
    # and the board contains no moves.
    #
    def reset(self):
        self.__board = np.zeros((3, 3),np.int8)
        self.__last_board = np.zeros((3, 3),np.int8)
        self.__game_over = False
        self.__draw = False
        self.__player = TicTacToe.__no_player
        self.__last_player = TicTacToe.__no_player
        
    #
    # Constructor has no arguments as it just sets the game
    # to an intial up-played set-up
    #
    def __init__(self):
        self.__board = np.zeros((3, 3),np.int8)
        self.__last_board = np.zeros((3, 3),np.int8)
        self.__game_over = False
        self.__draw = False
        self.__player = TicTacToe.__no_player
        self.__last_player = TicTacToe.__no_player
    
    #
    # return player as string "X" or "O"
    #
    def __player_to_str(self,player):
        if(player == TicTacToe.player_X): return "X"
        if(player == TicTacToe.player_O): return "O"
        return "?"
        
    #
    # Return a displayable version of the entire game.
    #
    def __str__(self):
        s = ""
        s += "Game Over: " + str(self.__game_over) +"\n"
        s += "Player :" + self.__player_to_str(self.__player) + "\n"
        s += "Current Board : \n" + str(self.__board)+ "\n"
        s += "Prev Player :" + self.__player_to_str(self.__last_player) + "\n"
        s += "Prev Current Board : \n" + str(self.__last_board)+ "\n"
        s += "State" + str(self.state()) + "\n"
        return s
    
    #
    # Return the list of valid actions
    #
    def list_actions(self):
        return list(TicTacToe.actions)
    
    #
    # Assume the move has been validated by move method
    # Make a copy of board before move is made and the last player
    #
    def __make_move(self, action, player):
        self.__last_board = np.copy(self.__board)
        self.__last_player = self.__player
        self.__player = player
        self.__board[TicTacToe.actions[action]] = player
        self.__game_over = self.__game_won()
        if(self.random_move() == None):
            self.__game_over = True
            self.__draw = True
        return
    
    #
    # Has a player already moved using the given action.
    #
    def __valid_move(self,action):
        return self.__board[TicTacToe.actions[action]] != self.__good_move
    
    #
    # If the proposed action is a valid move and the game is not
    # over. Make the given move (action) on behalf of the given 
    # player and update the game status
    #
    def move(self, action, player):
        if(self.__game_won()) : return TicTacToe.__bad_move_game_is_over
        if(self.__valid_move(action)): return TicTacToe.__bad_move_action_already_played 
        if(player == self.__player): return TicTacToe.__bad_move_no_consecutive_plays 
        
        self.__make_move(action,player)
        if(self.__game_over):
            if(self.__draw):
                return TicTacToe.__draw
            else:
                return TicTacToe.__win
        else:
            return TicTacToe.__play            

    #
    # Return (flattened) Game Ended, Last Player, Last Board, Player, Board
    #
    def detailed_state(self):
        flattened_state = []
        if(self.__game_over):
            flattened_state.append(1)
        else:
            flattened_state.append(0)
        flattened_state.append(self.__last_player)
        flattened_state.append(self.__player)
        for itm in np.reshape(self.__last_board,9).tolist() : flattened_state.append(itm)
        for itm in np.reshape(self.__board,9).tolist() : flattened_state.append(itm)
            
        return flattened_state

    #
    # Return state of current board as simple vector
    #
    def state(self,tostr=False):
        flattened_state = []
        for itm in np.reshape(self.__board,9).tolist() : flattened_state.append(itm)            
        if not tostr:
            return flattened_state
        else:
            return ''.join(str(e) for e in flattened_state)

    #
    # Show return the current board contents
    #
    def board(self):
        return self.__board
     
    #
    # Any row, column or diagonal with all player X or player O
    #
    def __game_won(self):
        rows = np.abs(np.sum(self.__board,axis=1))
        cols = np.abs(np.sum(self.__board,axis=0))
        diagLR = np.abs(np.sum(self.__board.diagonal()))
        diagRL = np.abs(np.sum(np.rot90(self.__board).diagonal()))
        
        if(np.sum(rows == self.__win_mask) > 0):
            return True
        if(np.sum(cols == self.__win_mask) > 0):
            return True
        if((np.mod(diagLR,3)) == 0) and diagLR > 0:
            return True
        if((np.mod(diagRL,3)) == 0) and diagRL > 0:
            return True
        return False

    #
    # Return which player goes next given the current player
    #
    def next_player(self,current_player):
        if(current_player == TicTacToe.player_O):
            return  TicTacToe.player_X
        else:
            return  TicTacToe.player_O
        
    #
    # Return a random action (move) that is still left
    # to make
    #
    def random_move(self):
        valid_moves = []
        for key in self.list_actions():
            if(self.__board[TicTacToe.actions[key]] == self.__good_move):
                valid_moves.append(key)
         
        num_poss_moves = len(valid_moves)
        if(num_poss_moves > 0):
            return valid_moves[randint(0, num_poss_moves-1)]
        else:
            return None
        
    #
    # Play a random game until completion.
    #
    def play(self):
        self.reset()
        plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Random player to start
        mv = None
        while(not self.__game_over):
            mv = self.random_move()
            self.move(mv,plyr)
            plyr = self.next_player(plyr)
        return self.__draw
    
    #
    # Run simulation to estimate Q values for state, action pairs. Random exploration policy
    # which should be tractable with approx 6K valid board states.
    #
    def estimate_Q_values(self,num_simulations):
        learning_rate0 = 0.05
        learning_rate_decay = 0.1
        discount_rate = 0.95
        reward = 0
        s = None
        sp = None
        Q = {}
        sim = 0 
        while(sim < num_simulations):
            self.reset()
            plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Random player to start
            mv = None
            while(not self.__game_over):
                s = self.state(tostr=True)
                mv = self.random_move()
                reward = self.move(mv,plyr)
                sp = self.state(tostr=True)
                learning_rate = learning_rate0 / (1 + (sim * learning_rate_decay))
                if s not in Q:
                    Q[s] = np.zeros((self.__num_actions))
                if sp not in Q:
                    Q[sp] = np.zeros((self.__num_actions))
                (Q[s])[mv-1] = learning_rate * (Q[s])[mv-1] + (1-learning_rate) * (reward + discount_rate * np.max(Q[sp]))
                plyr = self.next_player(plyr)
            if (sim % 100) == 0 : print(str(sim))
            sim += 1
        return Q
        

In [536]:
game = TicTacToe()
QV = game.estimate_Q_values(20000)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [537]:
print(QV.keys())

dict_keys(['1100-1-1000', '-101100-1-10', '-11100-110-1', '-10-11010-11', '0110-11-1-11', '-100-110001', '-1-1011-10-11', '1110-1-11-10', '0-10111-100', '001-101000', '01-100-1001', '10-10-11-100', '01-1001-110', '1000-100-11', '10-1-110100', '0101-1-1001', '101-10010-1', '0-101-11010', '1-10011-11-1', '100-100-110', '010-1-1-1110', '-101-1-11101', '0-11-10-11-11', '000011-1-10', '11-100010-1', '1-1-101-1-110', '-10010011-1', '011-110-1-1-1', '1-1-10-11-111', '0001-10-11-1', '001-11-1010', '-11-10-11110', '-10001-1-111', '01-1001-1-10', '1101-10-1-10', '0-1011100-1', '-101-11-1010', '11-11-10-1-10', '000-1-10110', '10-1101-100', '-11-1-101000', '-1-11-1-11010', '010-101000', '01-10-11-11-1', '-110010-11-1', '-1000101-10', '110-110-11-1', '1000-110-1-1', '11010-10-10', '10-1100-11-1', '0-11-100001', '-11-11-1010-1', '-1-110-10111', '0-10-11-1101', '1-11011-10-1', '01-10-1101-1', '00-110-1100', '01-101-1010', '-11100001-1', '1-1000-1011', '-101-11-1-110', '-11-110-101-1', '0011-1-11-10',

In [486]:
game = TicTacToe()
n = 0
go = True
while(go and n < 100):
    go = game.play()
    sys.stdout.write(".")
    n+=1
print("\n")
print(game.board())



..

[[ 0  1  1]
 [-1 -1 -1]
 [-1  0  1]]


In [460]:
print(game.board())
print(np.sum(game.board(),axis=0))


[[ 1  0 -1]
 [ 0 -1  1]
 [-1 -1  1]]
[ 0 -2  1]


In [471]:
bd =np.zeros((3, 3),np.int8)
pxm = np.full((3, 3),1,np.int8)
wn = np.full((1, 3),3,np.int8)
bd [0,0]=1
bd [0,1]=-1
bd [0,2]=-1
bd [1,0]=-1
bd [1,1]=1
bd [1,2]=-1
bd [2,0]=-1
bd [2,1]=1
bd [2,2]=-1
print(bd)
print((np.sum(np.abs(np.sum(bd,axis=0)) == wn) > 0))
print((np.sum(np.abs(np.sum(bd,axis=1)) == wn) > 0))

[[ 1 -1 -1]
 [-1  1 -1]
 [-1  1 -1]]
True
False


In [417]:
class testTicTacToe(unittest.TestCase):
    def test_one(self):
        self.assertEqual(1 == 1, True)

In [351]:
#
# Run A Single Test
#
test_to_run = "test_one"
suite = unittest.TestSuite()
suite.addTest(testTicTacToe(test_to_run))
runner = unittest.TextTestRunner()
runner.run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [352]:
#
# Run All Tests.
#
tests = testTicTacToe()
suite = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [353]:
game = TicTacToe()
print(game.list_actions())
print(game.board())
print(game.state())


[1, 2, 3, 4, 5, 6, 7, 8, 9]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
[0, -2, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [354]:
game.reset()
print(game.move(1,TicTacToe.player_X))
print(game.move(5,TicTacToe.player_X))
print(game.move(9,TicTacToe.player_X))

[[1 0 0]
 [0 0 0]
 [0 0 0]]
[0, -2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0
-3
-3


In [355]:
print(game.random_move())
print(game.random_move())
print(game.random_move())
print(game.random_move())
print(game.random_move())
print(game.random_move())
print(game.random_move())
print(game.random_move())

5
9
4
3
8
4
5
7


In [356]:
print(game)

Game Over: False
Player :X
Current Board : 
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Prev Player :?
Prev Current Board 
: [[0 0 0]
 [0 0 0]
 [0 0 0]]
State[0, -2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]



In [341]:
print(list(TicTacToe.actions))

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
class playTicTacToe:
    __actor_network_name = "net/actor"
    __critic_network_name = "net/critic"
    __x_units = 21

    def __init__(self):
        return
    
    #
    # Build a four layer fully connected Network.
    # 21 -> relu -> 21 -> relu -> 21 -> relu -> 21 -> relu
    #
    def construct_network(X_state,network_name):
        return